# Object Tracking Section Gaols

- Learn basic object tracking techniques
> - Optical Flow
> - MeanShift and CamShift
- Understand more advanced tracking
> - Review Built-in Tracking APIs
****

# 1. Optical Flow

- Optical Flow is the pattern of apparent motion of image objects between two consecutive frames caused by the movement of object on a camera.

- Optical Flow Analysis has a few assumptions:
> - The pixel intensities of an object do not change between consicutive frames.
> - Neighboring pixels have similar motion.

- The optical flow methods in OpenCV will first take in a given set of points and a frame. Then it will attempt to find those points in the next frame. It is up to the user to supply the points to track. 

- The Lucas-Kanade computes optical flow for a sparse feature set. Meaning only the points it was told to track. But what if we want to track all the points in a video?

- We can use Gunner Farneback's algorithm (also built-in to OpenCV) to calculate dense optical flow. This dense optical flow will calculate flow for all points in an image. It will color them black if no flow (no movement) is detected.

In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [2]:
corner_track_params = dict(
    maxCorners=10, 
    qualityLevel=0.3, 
    minDistance=7, 
    blockSize=7
)

lk_params = dict(
    winSize=(200, 200), 
    maxLevel=2,
    criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03)
)

In [3]:
cap = cv.VideoCapture(0)
ret, prev_frame = cap.read()

prev_gray = cv.cvtColor(prev_frame, cv.COLOR_BGR2GRAY)

# PTS TO TRACK
prev_pts = cv.goodFeaturesToTrack(prev_gray, mask=None, **corner_track_params)
mask = np.zeros_like(prev_frame)

while True:
    ret, frame = cap.read()
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    next_pts, status, err = cv.calcOpticalFlowPyrLK(prev_gray, 
                                                    frame_gray, prev_pts, None, **lk_params)
    
    good_new = next_pts[status==1]
    good_prev = prev_pts[status==1]
    
    for i, (new, prev) in enumerate(zip(good_new, good_prev)):
        x_new, y_new = new.ravel()
        x_prev, y_prev = prev.ravel()
        
        mask = cv.line(mask, (x_new, y_new), (x_prev, y_prev), (0, 255, 0), 3)
        
        frame = cv.circle(frame, (x_new, y_new), 8, (0, 0, 255), -1)
    
    img = cv.add(frame, mask)
    cv.imshow('tracking', img)
    k = cv.waitKey(30) & 0xFF
    if k == 27:
        break
        
    prev_gray = frame_gray.copy()
    prev_pts = good_new.reshape(-1, 1, 2)
    
cv.destroyAllWindows()
cap.release()

In [ ]:
cap = cv.VideoCapture(0)

ret, frame_1 = cap.read()
